# Phân tích và dự đoán giá bitcoin bằng Deep learning

In [13]:
from IPython.display import Image
Image(url= "https://media-cdn-v2.laodong.vn/storage/newsportal/2022/7/22/1071649/Bitcoin-23.jpg")

Bitcoin là một loại tiền tệ kỹ thuật số được phát hành dưới dạng phần mềm mã nguồn mở và phát minh bởi Satoshi Nakamoto. Đây cũng chính là đồng tiền điện tử đầu tiên trên thế giới, đặt nền móng cho sự phát triển của thị trường crypto. 

### Import thư viện cần thiết 

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt
import seaborn as sns
import plotly.graph_objects as px
from sklearn.preprocessing import MinMaxScaler
#Import những thư viện cho việc xử lý dữ liệu

In [15]:
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
#Import những thư viện cho việc dự đoán giá bitcoin

### Thu thập dữ liệu

In [16]:
crypto_currency = 'BTC'
against_currency = 'USD'
#Ta sẽ thao tác với dữ liệu giá Bitcoin (USD)

In [17]:
start = dt.datetime(2020, 1, 1) #Ngày bắt đầu của dữ liệu
end = dt.datetime.now()         #Ngày kết thúc (hôm nay)

In [18]:
data = web.DataReader(f'{crypto_currency}-{against_currency}','yahoo', start, end)
#Thu thập dữ liệu và để dưới dạng data frame

ConnectionError: HTTPSConnectionPool(host='finance.yahoo.com', port=443): Read timed out.

In [ ]:
data.head() 

Dữ liệu của chúng ta

In [ ]:
data.info()

In [ ]:
data.isna().sum()

Dữ liệu ko có thiếu

In [ ]:

plot = px.Figure(data=[px.Scatter(
    y=data['Close'],
    mode='lines'), 
])



plot.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                    step="day",
                    stepmode="backward"),
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
    )
)


 
plot.show()

### Tiền xử lý dữ liệu 

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))
#Chuẩn hóa dữ liệu data['Close'] theo Min-Max

In [ ]:
scaled_data

In [ ]:
prediction_days = 60
future_day = 30

In [ ]:
x_train, y_train= [],[]

In [ ]:
for x in range(prediction_days, len(scaled_data)-future_day):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x+future_day, 0])

In [ ]:

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


### Xây dựng mô hình dự đoán

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

In [ ]:
test_start = dt.datetime(2020,1,1)
test_end = dt.datetime.now()

In [ ]:
test_data = web.DataReader(f'{crypto_currency}-{against_currency}', 'yahoo', test_start, test_end)
actual_prices = test_data['Close'].values
total_dataset = pd.concat((data['Close'], test_data['Close']), axis = 0)
test_data

In [ ]:
model_inputs = total_dataset[len(total_dataset)-len(test_data)-prediction_days:].values
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.fit_transform(model_inputs)
model_inputs

In [ ]:
x_test = []
for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x, 0])

In [ ]:
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))


In [ ]:
prediction_prices = model.predict(x_test)
prediction_prices = scaler.inverse_transform(prediction_prices)

In [ ]:
prediction_prices

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
plt.plot(actual_prices, color ='black', label='Actual Prices')
plt.plot(prediction_prices, color ='green', label='Predicted Prices')
plt.title(f'{crypto_currency} price prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend(loc='upper left')
plt.show()

In [ ]:
real_data = [model_inputs[len(model_inputs) + 1 - prediction_days:len(model_inputs) + 1, 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

In [ ]:

prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print("Giá bitcoin ngày mai là:", prediction[-1][-1], "$")